In [1]:
# Ensure you have the following installed:
# numpy
# os
# compas
# compas_fd
# warnings
# traceback
# pickle

# Install by running:
# !pip install -r requirements.txt

In [2]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

### Step 1: Define the network

In [3]:
# Note that the coordinates of the vertices will change after equilibrium is reached.
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*10  # The coordinates of the vertices in mm
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.

scalar = 110
q_ring      = .5
q_connectors = .8
q_center    = .2
q           = np.array([q_connectors, q_center, q_center, q_connectors, q_connectors, q_center, q_center, q_connectors, q_ring, q_ring, q_ring, q_ring])/scalar
q[2]        = 1.5/scalar
q[-1]        = 1.5/scalar
fixed       = [0, 4, 5, 8]
# model_name  = 'VS_ring{}_connectors{}_center{}_as{}'.format(q_ring, q_connectors, q_center, q[2]*scalar)
model_name  = 'VS_ring{}_connectors{}_center{}_as{}_asr{}_s{}'.format(q_ring, q_connectors, q_center, q[2]*scalar, q[-1]*scalar, scalar)

# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)

# The topology of structure is fixed after generation, but you can update force densities after making the network
# q[0] = .10
# net.update_shape(q)

# Plot the network like this
net.net_plot(color=True, elables = True, vlabels = False)

(net.vertices == vertices).all() # Please understand that this is False
np.max(net.f)/0.078294515

# import matplotlib.pyplot as plt
# cmap = plt.get_cmap('viridis')

# Convert to TikZ RGB format
# f_plot = (net.f - np.min(net.f)) / (np.max(net.f) - np.min(net.f))
# for val in f_plot:
#     r, g, b, _ = cmap(val)  # Get RGBA (ignore A)color={rgb,1:red,0.282;green,0.141;blue,0.458}
#     print("\draw[thick, color = {" + f"rgb,1:red,{r:.3f}; green,{g:.3f};blue, {b:.3f}"+"}]")

np.max(net.f)/0.078294515

6.693023194619021

In [4]:
net2 = Network_custom.load_network(fr"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW\networks\VS_ring0.5_connectors0.8_center0.2_as1.5_asr1.5_s110_net.pkl")
# net2.net_plot(plot_type = 'arcs')

Here is the same network but elements 11 and 12 are combined into a single element.

In [5]:
# # Note that the coordinates of the vertices will change after equilibrium is reached.
# model_name  = 'Validation_structure_1'
# vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*10  # The coordinates of the vertices in mm
# edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
# paths       = [[0, 1, 2, 3], [4, 5, 6,7], [11, 8, 9, 10]]   # 3D printable paths. Ensure an edge is not flipped.
# directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.

# q_ring      = .7
# q_connectors = .5
# q_center    = .1
# q           = np.array([q_connectors, q_center, 2*q_center, q_connectors, q_connectors, q_center, q_center, q_connectors, q_ring, q_ring, q_ring, q_ring])/20
# fixed       = [0, 4, 5, 8]

# # Create the network
# net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)

# # The topology of structure is fixed after generation, but you can update force densities after making the network
# # q[0] = .10
# # net.update_shape(q)

# # Plot the network like this
# net.net_plot(color=True, elables = True, vlabels = False)

# (net.vertices == vertices).all() # Please understand that this is False

### Step 2: Materialize

In [6]:
# TPUc = {'E':130, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU conductive'} # Conductive TPU. Manufacturer Ninjatek Eel
TPU = {'E':77, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU non-conductive'} # Conductive TPU. Manufacturer Ninjatek Eel

file_path = os.path.join(BYU_UW_root, 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

net.set_material(TPU) # By saving the material properties in the network, we can easily see how the network was constructed.

# If all elements will get the same material, you can use the following line
# E = [TPU['E']]*len(net.edges)
A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
# l0, l_scalar = net.materialize(E, A)
# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)
net.l0, net.l_scalar

(array([62.57567703, 49.31307902, 12.24176822, 59.52452064, 55.83717107,
        40.71554575, 41.84097071, 61.0641256 , 54.62824288, 45.79647377,
        49.92209389, 27.83853488]),
 0.8684587410883032)

In [7]:
# # validate nl material method. This should be the same for the linear TPU
# strain_data = np.linspace(0, 0.5, 100)
# stress_data = TPU['E']*strain_data
# TPU_check = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU Check'}

# net.set_material(TPU_check) # By saving the material properties in the network, we can easily see how the network was constructed.
# l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
# net.l0, net.l_scalar

### Step 3: Define elements as a curves
An (optional) optimization algorithm finds the vertex locations minimizing abs(l1-l0). Next the (optimized) network is scaled down, and edges are defined as arcs.

available error functions in the optimization algorithms are:
- 'no optimization' : Network is scaled down directly, and no alogirthm is applied
- 'standard'        : a standard L2-norm quantifies the error
- 'sigmoid'         : a sigmoid function skews the L2-norm to penalize too short edges more (b>0) or too long edges more (b<0)

Preliminary results shows best results using a standard L2 error function

In [8]:
# net.initialize_shape_optimizer(function_type = 'no optimization',  method = 'L-BFGS-B', params = None)
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 1000, "damping": .1, "correction_scalar": 1.5, "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'trust-constr')
# net.initialize_shape_optimizer(function_type = 'sigmoid',  method = 'L-BFGS-B', params = {'a': 4, 'b': -1})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically

# #  You can plot the scaled network like this
# # net.net_plot(color=True, plot_type='scaled')

# # Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
# # for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net.arc_points(n = 10) 

net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)
# net.net_plot(color=True, plot_type='optimized', elables=False, vlabels = False)

# np.min(R/net.l0), net.l_scalar, np.min(net.l0 - net.l1), np.max(net.l0 - net.l1)
net.l_scalar

Iteration 0: Current error = 220.9324176196043
Iteration 100: Current error = 1.3850307731951104
Iteration 200: Current error = 1.3840614720817077
Iteration 300: Current error = 1.3778380051790835
Iteration 400: Current error = 1.378031800267642
Iteration 500: Current error = 1.3746449740168338
Iteration 600: Current error = 1.368693622632078
Iteration 700: Current error = 1.3620496468497016
Iteration 800: Current error = 1.3733620092013572
Iteration 900: Current error = 1.3633277553414573
Final error: 1.3598309497491945


0.9922795578598931

In [9]:
(net2.th * net2.R) - net2.l1_scaled

array([-1.42108547e-14,  9.40966790e-01,  9.59353733e-01,  0.00000000e+00,
        0.00000000e+00,  1.12607288e+00,  1.25986033e+00,  0.00000000e+00,
        7.66990177e-01,  2.08168701e-01,  0.00000000e+00,  3.29517443e-01])

In [ ]:
# import matplotlib.pyplot as plt
# n_tests = 20
# cs_vec = np.logspace(-1, np.log10(30), n_tests)
# reference_point = [0,0,0]                         # The network will be scaler relative to this point

# l_scalar_vec = []
# R_vec = []
# for cs in cs_vec:
#     net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 100000, "damping": .001, "correction_scalar": 1/cs, "tol": 1e-6})
#     net.optimize_vertices()
#     l_scalar_vec.append(net.l_scalar)
#     net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically
#     R, th = net.arc_param()
#     R_vec.append(R)
    
# fig, ax1 = plt.subplots(figsize=(12, 6))

# color = 'tab:blue'
# ax1.set_xlabel('alpha')
# ax1.set_ylabel('l_scalar', color=color)
# ax1.plot(cs_vec, l_scalar_vec, '*', color=color)
# ax1.tick_params(axis='y', labelcolor=color)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# color = 'tab:red'
# ax2.set_ylabel('R_min', color=color)
# for i in range(n_tests):
#     ax2.plot(cs_vec[i]*np.ones_like(R_vec[i]), R_vec[i], 'ko', color=color)
# ax2.set_ylim(0, 50)
# ax2.tick_params(axis='y', labelcolor=color)
# ax1.set_xscale('log')
# fig.tight_layout()  # to prevent overlap of labels
# plt.show()

In [ ]:

# reference_point = [0,0,0]                         # The network will be scaler relative to this point
# net.scale_vertices(reference_point, net.l_scalar, account_for_leafs=False) # If you don't provide a scalar, it will use network.l_scalar automatically. Note, that in this particular case, there are no leafs, so the last argument is not necessary.

# #  You can plot the scaled network like this
# # net.net_plot(color=True, plot_type='scaled')

# # Determine the Radius and Angle of the circle that define the arc of each element
# R, th = net.arc_param()
# # for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
# xyz = net.arc_points(n = 9) 

# net.net_plot(color=True, plot_type='arcs', elables=False)
# net.l_scalar, correction_scalar

In [ ]:
# %matplotlib qt
# import matplotlib.animation as animation
# ani = net.ShapeOptimizer.animate_optimization()
# writer = animation.FFMpegWriter(fps=15)
# ani.save(os.path.join(BYU_UW_root, 'images', 'form finding animations', f'form_finding_{model_name}_{0}.mp4'), writer=writer)

Flipping edge 2, 6, and 12 would make the network smoother

In [ ]:
# net.flip_curve(2, n = 10)          # Flip the curvature of one edge
net.auto_flip_curves(n = 10)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
net.flip_curves()                   # Flip all the edges.
net.flip_curves([2, 5, 8], n = 10)       # Flip the curvature of multiple edges
net.net_plot(color=True, plot_type='arcs', elables=True)

In [ ]:
# for edge_i, edge in enumerate(net.edges): (-3.13, 0.987) arc (239.60: 277.368:8.4696);
#     v0, v1 = edge
#     coor0, coor1 = net.vertices[v0], net.vertices[v1]
#     print(f'{edge_i}, starts at {coor0}, ends at {coor1}, and has a radius of {net.R[edge_i]} and an angle of {net.th[edge_i]}')
for edge_i, edge in enumerate(net.edges):
    v0, v1 = edge
    coor0, coor1 = net.vertices[v0], net.vertices[v1]
    R, th = net.R[edge_i], net.th[edge_i]
    center, theta0, theta1 = net._points_on_arc(coor0[:2], coor1[:2], R, print_parameters = True)
    # print(coor0[:2]/10, theta0*180/np.pi, theta1*180/np.pi, R)
    if R/10 > 10000:
        print(f"\draw ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) -- ({coor1[0]/10:.3f}, {coor1[1]/10:.3f});")
        continue
    if theta0 > theta1:
        print(f"\draw ({coor1[0]/10:.3f}, {coor1[1]/10:.3f}) arc ({theta0*180/np.pi:.3f}:{theta1*180/np.pi:.3f}:{-R/10:.3f});")
    else:
        print(f"\draw ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) arc ({theta0*180/np.pi:.3f}:{theta1*180/np.pi:.3f}:{R/10:.3f});")

### Step 4: Account for intersections. 
Curved paths and intersections are automatically set up in the backend

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
# When no interpolation function is provided, linear interpolation is used. No other interpolation function is implemented (yet?).
# net.jump_at_intersection(intersection_width = 2, intersection_height = 2, interpolation_function=None) # These settings are just for visibility, this would be way to much
# These setting are more appropriate for printing
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(color=True, plot_type='arcs', elables=True)

In [ ]:
model_name

Add loops to easily attach the printed network to a frame

In [ ]:
alpha_loop = np.deg2rad(30) # The angle of the loop
L_loop  = 5                # The length of the loop
n_points = 60               # The number of points in the loop

start_loop_bools = [True, True, False]
end_loop_bools = [True, True, False]

net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)

L_real = L_loop + L_loop*np.tan(alpha_loop)
L_real

Sometimes it takes a while before material starts extruding. This should preferably be fixed in the printer set up, but can me forced by adding a running start to a path

In [ ]:
start_loop_bools = [True, True, True]
end_loop_bools = [True, False, True]
L_loop  = 1
n_points = 5
net.add_running_start(start_loop_bools, end_loop_bools, L_loop, n_points)

In [ ]:
net.save_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net2.pkl'))

### Step 5: Generate G-code
Import and finalize printing parameters for materials and starting and end .gcode for the top. The start gcode has additional settings for temperatures and Linear Advance (K-factor). Linear advance is important for TPU as it is very visco-elastic. Linear advance can account for this

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
start_gcode     = open(r'DATA/start_gcode.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 65, 'first_layer_temperature':205, 'K-factor': 0.20}
g_code = replace_brackets(start_gcode, temperature_settings)

comment = ''

bed_width = 230
bed_height = 210

gen = G_code_generator(printing_params=printing_params)

printing_params['print_feedrate']

Make a Purge Line

In [ ]:
point0, point1 = [20,15,0.3], [90,15,0.3]
point2, point3 = [90,10,0.3], [20,10,0.3]
g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point0,
        point1=point1,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2.5,
        comment=comment)
g_code += gen.retract()
g_code += gen.move_to_point(point2[0:2], point2[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point2[0:2], point2[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point2,
        point1=point3,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2.5,
        comment=comment)
g_code += gen.retract()
g_code += gen.wipe(2 * np.pi) # Wipe the nozzle horizontally

Make the paths

In [ ]:
for path_i, cor_list in enumerate(net.paths_xyz):
    g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
    cor_list   = np.array(cor_list)
    # Move the coordinates to the center of the bed and add the layer height
    cor_list[:,0] += bed_width/2 -14
    cor_list[:,1] += bed_height/2 
    cor_list[:,2] += printing_params['layer_height']
    g_code += '\n'
    # Move the coordinates to the start point
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
    # Lower the nozzle
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
    # Unretract the filament
    g_code += gen.unretract()
    # Print the path
    for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
        g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor = printing_params['extrude_factor'],
            comment=comment)
    # Retract the filament
    g_code += gen.retract()
    # Wipe the nozzle
    g_code += gen.wipe_from_last_points(g_code)    
    # Raise the nozzle
    g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', model_name + '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)
print('G-code generated')

In [ ]:
minx, miny = 90, 90
maxx, maxy = 0, 0
for path_i, cor_list in enumerate(net.paths_xyz):
    cor_list   = np.array(cor_list)
    cor_list[:,0] += bed_width/2 -14
    cor_list[:,1] += bed_height/2 
    minx = min(minx, np.min(cor_list[:,0]))
    miny = min(miny, np.min(cor_list[:,1]))
    maxx = max(maxx, np.max(cor_list[:,0]))
    maxy = max(maxy, np.max(cor_list[:,1]))

minx , miny , maxx , maxy 